In [2]:
import sqlite3
from sklearn.datasets import fetch_lfw_people
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

# load the data using sklearn
lfw_dataset = fetch_lfw_people(data_home='./LFW/', min_faces_per_person=20, download_if_missing=True, color=True)

# connecting to our SQLite database
conn = sqlite3.connect('lfw_dataset.db')
cursor = conn.cursor()

# creating a table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS faces (
        id INTEGER PRIMARY KEY,
        target INTEGER,
        name TEXT NOT NULL,
        image BLOB NOT NULL
    )
''')

image_index = -1
# Inserting the targets, names, and images into the table
for images in lfw_dataset.images:
    # Convert the image data to bytes
    image_bytes = pickle.dumps(images)
    image_index += 1
    # Get the target index for the specified image
    target_index = lfw_dataset.target[image_index]
    # Get the corresponding name from target_names
    name = lfw_dataset.target_names[target_index]
    # Insert the record into the database with target as id
    cursor.execute("INSERT INTO faces (target, name, image) VALUES (?, ?, ?)", (int(target_index), name, image_bytes))

# commit the changes and close the connection
conn.commit()
conn.close()

# printing the rows in the dataset to see if it works fine (just for testing)
conn = sqlite3.connect('lfw_dataset.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM faces')

rows = cursor.fetchall()

if not rows:
    print("No data.")
else:
    print(f"Total rows: {len(rows)}")
    # Print the data
    for row in rows:
        print(f"Labels: {row[1]}, Name: {row[2]}")

# Close the connection
conn.close()


Total rows: 3023
Labels: 61, Name: Winona Ryder
Labels: 25, Name: Jean Chretien
Labels: 9, Name: Carlos Menem
Labels: 5, Name: Ariel Sharon
Labels: 1, Name: Alvaro Uribe
Labels: 10, Name: Colin Powell
Labels: 48, Name: Recep Tayyip Erdogan
Labels: 17, Name: Gray Davis
Labels: 13, Name: George Robertson
Labels: 54, Name: Silvio Berlusconi
Labels: 14, Name: George W Bush
Labels: 14, Name: George W Bush
Labels: 45, Name: Nestor Kirchner
Labels: 25, Name: Jean Chretien
Labels: 8, Name: Bill Clinton
Labels: 14, Name: George W Bush
Labels: 9, Name: Carlos Menem
Labels: 1, Name: Alvaro Uribe
Labels: 14, Name: George W Bush
Labels: 10, Name: Colin Powell
Labels: 12, Name: Donald Rumsfeld
Labels: 14, Name: George W Bush
Labels: 21, Name: Hugo Chavez
Labels: 14, Name: George W Bush
Labels: 14, Name: George W Bush
Labels: 14, Name: George W Bush
Labels: 14, Name: George W Bush
Labels: 3, Name: Andre Agassi
Labels: 15, Name: Gerhard Schroeder
Labels: 47, Name: Pete Sampras
Labels: 5, Name: Ariel S